# 0.0 IMPORTS

In [32]:
!pip install keras
!pip install tensorflow

     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ------------ --------------------------- 0.5/1.7 MB 16.0 MB/s eta 0:00:01
     --------------------------------- ------ 1.5/1.7 MB 18.4 MB/s eta 0:00:01
     ---------------------------------------- 1.7/1.7 MB 13.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/272.8 MB ? eta -:--:--
     ---------------------------------------- 0.2/272.8 MB 5.3 MB/s eta 0:00:52
     ---------------------------------------- 0.3/272.8 MB 3.9 MB/s eta 0:01:10
     ---------------------------------------- 0.5/272.8 MB 4.2 MB/s eta 0:01:06
     ---------------------------------------- 0.7/272.8 MB 3.9 MB/s eta 0:01:10
     ---------------------------------------- 0.9/272.8 MB 4.3 MB/s eta 0:01:04
     ---------------------------------------- 1.1/272.8 MB 4.1 MB/s eta 0:01:07
     ---------------------------------------- 1.3/272.8 MB 4.1 MB/s eta 0:01:07
     ---------------------------------------- 1.5/272.8 MB 

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'P:\\Python\\GitHub\\airbnb_prediction\\airbnb_env\\Lib\\site-packages\\~umpy\\.libs\\libopenblas64__v0.3.21-gcc_10_3_0.dll'
Check the permissions.



In [30]:
!pip install scikit-learn

     ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
     ----- ---------------------------------- 1.2/8.3 MB 37.3 MB/s eta 0:00:01
     -------------- ------------------------- 3.0/8.3 MB 38.2 MB/s eta 0:00:01
     ---------------------- ----------------- 4.7/8.3 MB 37.3 MB/s eta 0:00:01
     ------------------------------ --------- 6.2/8.3 MB 36.2 MB/s eta 0:00:01
     -------------------------------- ------- 6.8/8.3 MB 31.1 MB/s eta 0:00:01
     ------------------------------------ --- 7.5/8.3 MB 28.2 MB/s eta 0:00:01
     ------------------------------------- -- 7.9/8.3 MB 25.2 MB/s eta 0:00:01
     ---------------------------------------  8.3/8.3 MB 23.1 MB/s eta 0:00:01
     ---------------------------------------  8.3/8.3 MB 23.1 MB/s eta 0:00:01
     ---------------------------------------- 8.3/8.3 MB 19.0 MB/s eta 0:00:00
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached scipy-1.10

## packages

In [40]:
import os, pickle, datetime, json
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import models as ml
from keras import layers as l


pd.options.display.max_columns = 500 # this will set limit of columns to 500
pd.set_option('display.float_format', lambda x: '%.5f' % x)

%matplotlib inline
os.getcwd()

ModuleNotFoundError: No module named 'tensorflow'

## 0.1 Helper Function

## 0.2 Loading Data

In [8]:
df0 = pd.read_csv("data/train_users_2.csv")
df0.columns = df0.columns.str.lower()
df0.head(3)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.00000,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.00000,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US


In [10]:
df_sessions = pd.read_csv( "data/sessions.csv" )
df_sessions.columns = df_sessions.columns.str.lower()
df_sessions.head(3)

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.00000
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.00000
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.00000


# 1.0 Data Description

In [15]:
df1 = df0.copy()

In [16]:
df1.shape

(213451, 16)

In [17]:
df1.isnull().sum()/len(df1)

id                        0.00000
date_account_created      0.00000
timestamp_first_active    0.00000
date_first_booking        0.58347
gender                    0.00000
age                       0.41223
signup_method             0.00000
signup_flow               0.00000
language                  0.00000
affiliate_channel         0.00000
affiliate_provider        0.00000
first_affiliate_tracked   0.02841
signup_app                0.00000
first_device_type         0.00000
first_browser             0.00000
country_destination       0.00000
dtype: float64

In [19]:
print( len(df1) )
df1 = df1.dropna()
print( len(df1) )

213451
68171


In [18]:
df1.dtypes

id                          object
date_account_created        object
timestamp_first_active       int64
date_first_booking          object
gender                      object
age                        float64
signup_method               object
signup_flow                  int64
language                    object
affiliate_channel           object
affiliate_provider          object
first_affiliate_tracked     object
signup_app                  object
first_device_type           object
first_browser               object
country_destination         object
dtype: object

In [23]:
for i in ["date_account_created", "date_first_booking"]:
    df1[i] = pd.to_datetime(df1[i])
df1[ 'timestamp_first_active' ] = pd.to_datetime( df1['timestamp_first_active'], format="%Y%m%d%H%M%S" )
df1['age'] = df1['age'].astype(int)
print( df1.dtypes )

id                                 object
date_account_created       datetime64[ns]
timestamp_first_active     datetime64[ns]
date_first_booking         datetime64[ns]
gender                             object
age                                 int32
signup_method                      object
signup_flow                         int64
language                           object
affiliate_channel                  object
affiliate_provider                 object
first_affiliate_tracked            object
signup_app                         object
first_device_type                  object
first_browser                      object
country_destination                object
dtype: object


# 2.0 Data Filtering

In [24]:
df2 = df1.copy()

# 3.0 Data Prep

In [25]:
df3 = df2.copy()

# 4.0 Feature Selection

In [27]:
cols_drop = ['date_account_created', 'timestamp_first_active', "date_first_booking"]
df4 = df3.drop( cols_drop , axis=1)

In [38]:
X = df4.drop( ["country_destination","id"] , axis=1 )
y = df4['country_destination'].copy()

In [39]:
df4.sample().T

,81138
id,buaq6yt9xd
gender,FEMALE
age,28
signup_method,facebook
signup_flow,0
language,en
affiliate_channel,other
affiliate_provider,other
first_affiliate_tracked,tracked-other
signup_app,Web


# 5.0 Neural Network - Multi-Layer Perceptron

In [37]:
Xtrain, Xtest, ytrain, ytest = train_test_split( X, y, test_size = 0.2, random_state=42 )

In [ ]:
model = ml.Sequential()
model.add( 128, input_dim = Xtrain.shape[0], activation = "relu")

# 6.0 NN Performance